In [49]:
#IMPORTS
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV #splits up your test set in eqally sized parts, uses one part as test data and the rest as training data. So it optimizes as many classifiers as parts you split your data into.

from sklearn.neighbors import KNeighborsClassifier


In [42]:
#LOAD DATA

df = pd.read_csv('USER-FEATURE-DATA.csv')
current_row=0
df_sample = pd.DataFrame([],columns=df.columns)
for i in np.unique(df['user1']):
      new_sample=df[df['user1']==i].sample(n=5)
      df_sample=pd.concat([df_sample,new_sample],axis=0)

df_sample = df_sample.reset_index(drop=True)

y = df_sample.iloc[:, 0]
X = df_sample.iloc[:, 1:]
print(y[5]) 

user19


In [ ]:
#K NEAREST NEIGHBOURS
clf = KNeighborsClassifier(n_neighbors=1, metric='euclidean',)

In [48]:
#TRAINING

kf = KFold(n_splits=5)

ATTRIBUTES = len(X.columns)
print('n attrb =', ATTRIBUTES)
USERS = y.nunique()
print('n users =', USERS)
neurons = (ATTRIBUTES + USERS) // 2 
clf = MLPClassifier(hidden_layer_sizes=(neurons,), solver='lbfgs', momentum=0.2, learning_rate_init=0.3, random_state=0)
clf.n_layers_ = 3
clf.max_iter = 1000
#got rid of:
#hidden_layer_sizes=(1,) (scores all zeroes)
#  l
#
# look into RandomSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd'],
            'hidden_layer_sizes': [
              (neurons-10,), (neurons-5,), (neurons,), (neurons+5,), (neurons+10,)
             ],
             'momentum': [0.1,0.2,0.3],
             'learning_rate_init':[0.2,0.3,0.4]
        }
       ]

GRID_SEARCH_TEST = False 

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    if GRID_SEARCH_TEST:
        clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
        clf.fit(X_train, y_train)
        print('params:', clf.best_params_)
    else:
      clf.fit(X_train, y_train)
      print('SCORE:', clf.score(X_test, y_test))
    
    #EXTRA INFO:
     

      print('current loss computed with the loss function: ',clf.loss_)
      #print('coefs: ', clf.coefs_)
      print('intercepts: ',clf.intercepts_)
      print(' number of iterations the solver: ', clf.n_iter_)
      print('num of layers: ', clf.n_layers_)
      print('Num of o/p: ', clf.n_outputs_) 
      
    



n attrb = 102
n users = 20
SCORE: 0.0
current loss computed with the loss function:  0.006442355626473984
intercepts:  [array([-3.84882981e-03, -1.21879592e-01, -4.89825291e-01, -8.05099232e-01,
       -1.90145700e-01,  2.73969060e+00, -8.30891386e-02,  5.95695786e+00,
        5.92727222e+00,  2.35821677e+00, -1.45456970e-01,  4.15435173e+00,
        1.04175361e-01, -4.98121199e+00,  4.07806148e+00, -7.59553739e-01,
       -6.97788549e-02, -2.23285544e+00, -3.64136189e+00, -9.40244178e-01,
        2.32903494e+00,  1.08773640e+01,  4.45699246e+00,  6.92439413e-02,
        2.42211940e+00,  9.97957577e-02,  8.18764664e-01,  3.17789980e+00,
        1.91059006e+00,  1.66763062e+00, -1.19808892e-01, -8.05294237e-01,
       -2.49423720e+00,  5.99842395e+00, -7.60285744e-01,  8.69340066e-02,
       -2.07182623e+00, -3.65509794e+00,  1.64296514e+00, -1.16310705e-01,
        6.96323106e+00,  5.36157381e-02,  1.68067351e+00, -7.55657970e-02,
        1.86380325e+00,  2.88617683e+00, -1.41755360e+0